# Exploratory Data Analysis of Heavy Truck J1939 data

Ce notebook va servir de base dans l'analyse des données du `Heavy Truck`. Ces données reprennent une payload que nous pouvons retrouver avec le protocole **bus CAN J1939**.

**Payload d'un message sous protocole J1939:**

![payload j1939](./images/j1939-payload.png)

:warning: Attention à bien écouter Vendredi 26 la présentation de `Simon Bellemare` sur les données **bus CAN J1939** !

## Imports

In [ ]:
from pathlib import Path

import dask.dataframe as dd
import pandas as pd

## Chargement des données

Il est important de noter que l'arborescence des données est la suivante:

```txt
|--- heavy_truck_data
    |--- part_1
        |--- part_1
            |--- csv
    |--- part_2
        |--- part_2
            |--- csv
    |--- part_3
        |--- part_3
            |--- csv
    |--- part_4
        |--- part_4
            |--- csv
```

TODO: Vérifier la confirmité de tous les dataframes (colonnes, separator, etc...)

In [ ]:
ddf = dd.read_csv("../data/heavy_truck_data/**/*.csv", sep=";", on_bad_lines="skip")

## Découverte des données

### Les colonnes

Nous pouvons commencer par regarder quelles colonnes composent nos données. Cela nous donnera peut être une idée des parties du message qui sont conservés.

In [ ]:
ddf.columns

Il y a donc 3 parties claires que nous possédons dans les données:

- `id`: l'adresse de l'émetteur, du récepteur, de la priorité et/ou autre (voir la capture d'écran: PGN par exemple).
- `dlc`: le fameux Data Length Content qui indique la taille du payload.
- `data`: ce doit être la payload en elle même.

Il serait intéressant de confirmer nos intuitions par rapport au graphique du payload montré en début de notebook.

In [ ]:
ddf.dtypes

On a que des valeurs décimales, nous allons soit devoir les convertir en binaire soit les garders en décimales, excepté la colonne `timestamp`.

### Les valeurs d'ID

In [ ]:
ddf["id"].head(10)

Ce sont des valeurs réelles, sous 256 valeurs ? Vérifions.

In [ ]:
ddf["id"].min().compute(), ddf["id"].max().compute()